## Mana-fest Destiny

In [5]:
%load_ext autoreload
%autoreload 2

#Imports
import os
import pandas as pd
import requests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Clean Slate wooo

In [6]:


from scripts.get_card_data import get_card_data
#import importlib
#importlib.reload(get_card_data)

# Fetch card data for Bloomburrow (should be available)
print("Fetching Bloomburrow card data...")
blb_data = get_card_data("blb")
print(blb_data.head())  # Show the first few rows

# Fetch card data for Duskmourne (assuming it's not preloaded)
print("\nFetching Duskmourne card data...")
dsk_data = get_card_data("dsk")
print(dsk_data.head())

# Fetch Duskmourne again to confirm it's not being re-fetched
print("\nFetching Duskmourne again...")
dsk_data_again = get_card_data("dsk")

# Verify the dataframes are the same object (not refetched)
assert dsk_data is dsk_data_again, "Duskmourne data was refetched instead of being cached."
print("Confirmed: Duskmourne card data was retrieved only once.")


Fetching Bloomburrow card data...
Loading Scryfall data for Bloomburrow from existing file...
  object                                    id  \
0   card  7dd9946b-515e-4e0d-9da2-711e126e9fa6   
1   card  39ebb84a-1c52-4b07-9bd0-b360523b3a5b   
2   card  b24c3e9c-2aff-45fd-9c87-3ebd33bbe807   
3   card  436d6a84-4cea-4ca7-94aa-9d08280652af   
4   card  d3871fe6-e26e-4ab4-bd81-7e3c7b8135c1   

                              oracle_id multiverse_ids   mtgo_id  arena_id  \
0  7b89b7d2-c724-4d5d-9f0b-7d3302ad1168       [669036]  129489.0   91658.0   
1  381a3e8e-71dd-48e4-ab62-53478bde4a14       [668996]  129409.0   91618.0   
2  eb5406e1-c3eb-4434-91f4-308622d3eab4             []       NaN       NaN   
3  63d50d5e-56de-44c4-b906-b2a21a56e3f3       [669118]  129653.0   91740.0   
4  ce36a450-68a6-4519-a9f0-0e3b772054eb       [669037]  129491.0   91659.0   

   tcgplayer_id  cardmarket_id                     name lang  ...  \
0      559491.0       778435.0            Agate Assault   en  ...  